# Modelling

### Contents

- [Code](#Code)
- [Results](#Results)

Here, we run the module that will be used for player prediction and squad generation. We will use the pycaret classification module. In this code, we assign the x values (player stats) and y values (binary recommendations).

We then split the the dataset into a 80/20 train:test split.

The 11 models (1 for each player role) are then saved. These model files are then loaded onto the streamlit app for prediction and squad generation.

## Code

In [10]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, load_model, finalize_model, save_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Assuming the last columns are the target labels and the rest are features
X = df.iloc[:, 6:20]  # Select columns 7 to 20 (0-based index: 6 to 19)
y = df.iloc[:, -11:]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of target column names
target_columns = y.columns

# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['target', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

for target in target_columns:
    X_target_train = X_train.copy()
    X_target_test = X_test.copy()
    y_target_train = y_train[target]
    y_target_test = y_test[target]
    df_target = pd.concat([X_target_train, y_target_train], axis=1)

    # Setup PyCaret environment
    clf = setup(data=df_target, target=target, session_id=42)

    # Compare different models
    best_model = compare_models()
    
    # Get the original results from compare_models, which includes 'TT'
    compare_results = pull()

    # Mapping from scikit-learn class names to PyCaret's model names
    model_name_mapping = {
        'LogisticRegression': 'Logistic Regression',
        'KNeighborsClassifier': 'K Neighbors Classifier',
        'DecisionTreeClassifier': 'Decision Tree Classifier',
        'RidgeClassifier': 'Ridge Classifier',
        'AdaBoostClassifier': 'Ada Boost Classifier',
        'GradientBoostingClassifier': 'Gradient Boosting Classifier',
        'LinearDiscriminantAnalysis': 'Linear Discriminant Analysis',
        'ExtraTreesClassifier': 'Extra Trees Classifier',
        'XGBClassifier': 'Extreme Gradient Boosting',
        'GaussianNB': 'Naive Bayes',
        'SVC': 'SVM - Linear Kernel',
        'RandomForestClassifier': 'Random Forest Classifier',
        'LGBMClassifier': 'Light Gradient Boosting Machine',
        'QuadraticDiscriminantAnalysis': 'Quadratic Discriminant Analysis',
        'DummyClassifier': 'Dummy Classifier'
    }

    # Extract the model name
    model_name = type(best_model).__name__
    pycaret_model_name = model_name_mapping.get(model_name)

    # Extract testing time (TT) from compare_models results by matching the PyCaret model name
    if pycaret_model_name:
        testing_time_row = compare_results[compare_results['Model'] == pycaret_model_name]
        if not testing_time_row.empty:
            testing_time = testing_time_row['TT (Sec)'].values[0]
        else:
            print(f"No match found for model {pycaret_model_name} in compare_results.")
            testing_time = None
    else:
        print(f"No mapping found for model {model_name}.")
        testing_time = None

    # Finalize the best model
    final_model = finalize_model(best_model)

    # Predict on the test set
    predictions_test = predict_model(final_model, data=X_target_test)
    predictions_train = predict_model(final_model, data=X_target_train)

    # Calculate test accuracy and F1 score
    y_pred_test = predictions_test['prediction_label']
    test_accuracy = accuracy_score(y_target_test, y_pred_test)
    test_f1 = f1_score(y_target_test, y_pred_test, average='weighted')

    # Calculate train accuracy and F1 score
    y_pred_train = predictions_train['prediction_label']
    train_accuracy = accuracy_score(y_target_train, y_pred_train)
    train_f1 = f1_score(y_target_train, y_pred_train, average='weighted')

    # Append results to DataFrame
    new_row = pd.DataFrame({'target': target, 
                            'train_accuracy': train_accuracy, 
                            'test_accuracy': test_accuracy, 
                            'train_f1': train_f1, 
                            'test_f1': test_f1, 
                            'TT': testing_time},index=[0])
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Save the model
    save_model(final_model, f'model_{target}')

    # Load the model for future predictions
    loaded_model = load_model(f'model_{target}')

# Save the results to a PKL file using joblib
joblib.dump(results_df, 'finalmodel.pkl')


,Description,Value
0,Session id,42
1,Target,Class_Traditional Keeper
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9833,0.9000,0.9000,0.8500,0.8667,nan,0.8632,0.0160
dt,Decision Tree Classifier,0.9833,0.8900,0.9000,0.8500,0.8667,nan,0.8632,0.0120
ridge,Ridge Classifier,0.9833,0.8800,0.9000,0.8500,0.8667,nan,0.8632,0.0120
rf,Random Forest Classifier,0.9833,0.9000,0.8000,0.8000,0.8000,nan,0.8000,0.0640
lda,Linear Discriminant Analysis,0.9833,0.8800,0.9000,0.8500,0.8667,nan,0.8632,0.0110
ada,Ada Boost Classifier,0.9667,0.8900,0.8000,0.7500,0.7667,nan,0.7632,0.0420
gbc,Gradient Boosting Classifier,0.9667,0.8900,0.8000,0.7500,0.7667,nan,0.7632,0.0350
et,Extra Trees Classifier,0.9667,0.9000,0.8000,0.7500,0.7667,nan,0.7632,0.0540
xgboost,Extreme Gradient Boosting,0.9667,0.9000,0.8000,0.7500,0.7667,nan,0.7632,0.0170
lr,Logistic Regression,0.9524,0.8800,0.8000,0.7000,0.7333,nan,0.7278,0.0210


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

Index(['Model', 'Accuracy', 'AUC', 'Recall', 'Prec.', 'F1', 'Kappa', 'MCC',
       'TT (Sec)'],
      dtype='object')
Best model name: KNeighborsClassifier
Mapped PyCaret model name: K Neighbors Classifier


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9690,1.0000,1.0000,0.9550,0.9746,0.9362,0.9437,0.0700
xgboost,Extreme Gradient Boosting,0.9690,0.9889,1.0000,0.9550,0.9746,0.9362,0.9437,0.0240
et,Extra Trees Classifier,0.9548,1.0000,1.0000,0.9350,0.9635,0.9058,0.9168,0.0620
knn,K Neighbors Classifier,0.9381,0.9681,1.0000,0.9100,0.9492,0.8725,0.8875,0.0170
ada,Ada Boost Classifier,0.9214,0.9889,0.9500,0.9350,0.9349,0.8320,0.8527,0.0430
lr,Logistic Regression,0.9190,0.9556,0.9750,0.9150,0.9353,0.8362,0.8592,0.0160
gbc,Gradient Boosting Classifier,0.9048,0.9167,0.9500,0.9100,0.9206,0.8058,0.8289,0.0440
lightgbm,Light Gradient Boosting Machine,0.8929,0.9722,0.9167,0.9017,0.9052,0.7808,0.7918,0.0360
ridge,Ridge Classifier,0.8857,0.9556,0.9083,0.8967,0.8984,0.7625,0.7716,0.0120
lda,Linear Discriminant Analysis,0.8857,0.9556,0.9083,0.8967,0.8984,0.7625,0.7716,0.0100


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

Index(['Model', 'Accuracy', 'AUC', 'Recall', 'Prec.', 'F1', 'Kappa', 'MCC',
       'TT (Sec)'],
      dtype='object')
Best model name: RandomForestClassifier
Mapped PyCaret model name: Random Forest Classifier


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


['finalmodel.pkl']

## Results

The average model accuracy is 0.9485, indicating that our predictor will predict the roles correctly at a high level.

| Average Model Accuracy  | 0.9485 |
|-------------------------|--------|

There is on the whole a very high train/test accuracy and f1 for all the top models taken for each role.

In [11]:
#Display the train/test accuracy and F1, and time taken
results_df

,target,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Class_Traditional Keeper,0.978022,1.000000,0.978664,1.000000,0.016
1,Class_Sweeper Keeper,1.000000,1.000000,1.000000,1.000000,0.025
2,Class_Ball Playing Defender,1.000000,0.956522,1.000000,0.960339,0.102
3,Class_No Nonsense Defender,1.000000,0.956522,1.000000,0.954694,0.023
4,Class_Full Back,1.000000,0.956522,1.000000,0.952704,0.036
5,Class_All Action Midfielder,1.000000,0.869565,1.000000,0.875049,0.024
6,Class_Midfield Playmaker,1.000000,1.000000,1.000000,1.000000,0.021
7,Class_Traditional Winger,1.000000,1.000000,1.000000,1.000000,0.031
8,Class_Inverted Winger,1.000000,0.869565,1.000000,0.870062,0.024
9,Class_Goal Poacher,0.923077,0.869565,0.922790,0.864803,0.026


The average train accuracy and F1 lies at around 0.99, while test accuracy and F1 lies at just below 0.93. Time taken averages at 0.03secs, which is very fast.

| train_accuracy | test_accuracy | train_f1 | test_f1 | TT     |
|----------------|---------------|----------|---------|--------|
| 0.9910         | 0.9289        | 0.9910   | 0.9275  | 0.0362 |

Over here, we can see that accuracy of the top 3 models for each role are all at a very high level, with the lowest being 0.8524. 

| Role                        |   M1   |   M2   |   M3   |
|-----------------------------|:------:|:------:|:------:|
| Class_Traditional Keeper    | 0.9857 | 0.9857 | 0.9857 |
| Class_Sweeper Keeper        | 0.9381 | 0.9214 | 0.9214 |
| Class_Ball Playing Defender | 0.9714 | 0.9690 | 0.9690 |
| Class_No Nonsense Defender  | 0.9548 | 0.9524 | 0.9524 |
| Class_Full Back             | 0.9357 | 0.9048 | 0.9048 |
| Class_All Action Midfielder | 0.9333 | 0.9214 | 0.9214 |
| Class_Midfield Playmaker    | 0.9405 | 0.9405 | 0.9405 |
| Class_Traditional Winger    | 0.9690 | 0.8905 | 0.8905 |
| Class_Inverted Winger       | 0.9500 | 0.9381 | 0.9190 |
| Class_Goal Poacher          | 0.9381 | 0.8548 | 0.8524 |
| Class_Target Man            | 0.9167 | 0.9095 | 0.8857 |

For greater clarity, we can see the top 3 models that produced the accuracy above for each role. Most of the top models were Logistic Regression, while 2 were K Neighbors Classifier, and 1 each were Extra Trees Classifier and Random Forest Classifier.

|             Role            |        1st Model       |        2nd Model       |          3rd Model         |
|:---------------------------:|:----------------------:|:----------------------:|:--------------------------:|
| Class_Traditional Keeper    | KNeighborsClassifier   | DecisionTreeClassifier | RidgeClassifier            |
| Class_Sweeper Keeper        | LogisticRegression     | ExtraTreesClassifier   | GaussianNB                 |
| Class_Ball Playing Defender | ExtraTreesClassifier   | RidgeClassifier        | LinearDiscriminantAnalysis |
| Class_No Nonsense Defender  | LogisticRegression     | ExtraTreesClassifier   | LGBMClassifier             |
| Class_Full Back             | LogisticRegression     | RidgeClassifier        | ExtraTreesClassifier       |
| Class_All Action Midfielder | LogisticRegression     | KNeighborsClassifier   | RidgeClassifier            |
| Class_Midfield Playmaker    | LogisticRegression     | ExtraTreesClassifier   | XGBClassifier              |
| Class_Traditional Winger    | LogisticRegression     | RidgeClassifier        | LinearDiscriminantAnalysis |
| Class_Inverted Winger       | LogisticRegression     | RidgeClassifier        | LinearDiscriminantAnalysis |
| Class_Goal Poacher          | KNeighborsClassifier   | ExtraTreesClassifier   | LogisticRegression         |
| Class_Target Man            | RandomForestClassifier | XGBClassifier          | ExtraTreesClassifier       |

For further breakdowns on how all models performed on each role, please refer to 02a-modellingappendix